## Overview

In this notebook, we load a saved model trained in `example_model_training.ipynb`, and use it to perform predictive 
inference on events of interest from frames from a new video it has not been trained on.

In [ ]:
# Load saved model
# Pass all frames from video through, get rounded predictions for each frame
# Convert these predictions to text format: ("Possession: True / False", "Set Piece: True / False")
# Overlay text on video frame-by-frame, save new output video